In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip3 install nlpaug
import tarfile
from wordcloud import WordCloud, STOPWORDS
import re
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint 
import datetime

In [2]:
import tarfile
fname='D:/sentence correction 2/fce_v2.1.bea19.tar.gz'
fname.endswith("tar.gz")
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [3]:
def m2_to_txt(path):
  '''
  this function processes the .m2 format file 
  and converts the required sentence into txt format'''
  m2 = open(path).read().strip().split("\n\n")
  corrupted = open('corrupted.txt', "w")
  uncorrupted=open('uncorrupted.txt','w')
  skip = {"noop", "UNK", "Um"}
  for sent in m2:
    sent=sent.split('\n')
  
    incorr_sent=sent[0].split()[1:]
  
    corrupted.write(" ".join(incorr_sent)+"\n")
    edits = sent[1:]
  
    offset=0
  
    for edit in edits:
        edit = edit.split("|||")
        if edit[1] in skip: continue # Ignore certain edits
        
        
        span = edit[0].split()[1:] # Ignore "A "
        start = int(span[0])
        end = int(span[1])
        cor = edit[2].split()
        incorr_sent[start+offset:end+offset] = cor
    
        offset = offset-(end-start)+len(cor)
      
    uncorrupted.write(" ".join(incorr_sent)+"\n")

In [4]:
m2_to_txt('D:/sentence correction 2/fce/m2/fce.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data1=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('D:/sentence correction 2/fce/m2/fce.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data2=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('D:/sentence correction 2/fce/m2/fce.test.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data3=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])



In [5]:
data=pd.concat([data1, data2, data3],ignore_index=True)

In [6]:
data=data[data['corrupted']!=data['uncorrupted']]#removing rows which have same values in both columns

In [7]:
fname='sm_norm_mt.tar.gz'
fname.endswith("tar.gz")
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [8]:
with open('D:/sentence correction 2/en2cn-2k.en2nen2cn','r') as file:
  line=file.readlines()
corrupted=[]
uncorrupted=[]
for i,lines in enumerate(line):
  if i%3==0:
    corrupted.append(lines.strip())
  elif (i%3==1):
    uncorrupted.append(lines.strip())
  else:
    pass

In [9]:
data4=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

In [10]:
data= data.sample(frac=1).reset_index(drop=True)

In [11]:
data=data.drop_duplicates()

In [12]:

def decontractions(phrase):
  """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""

  # specific
  phrase = re.sub(r"won\'t", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)
  phrase = re.sub(r"won\’t", "will not", phrase)
  phrase = re.sub(r"can\’t", "can not", phrase)

  # general
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)

  return phrase

In [13]:
data['corrupted']=data['corrupted'].apply(lambda x:decontractions(x))
data['uncorrupted']=data['uncorrupted'].apply(lambda x:decontractions(x))

data4['corrupted']=data4['corrupted'].apply(lambda x:decontractions(x))
data4['uncorrupted']=data4['uncorrupted'].apply(lambda x:decontractions(x))

In [14]:
data['corrupted']=data['corrupted'].apply(lambda x:x.lower())
data['uncorrupted']=data['uncorrupted'].apply(lambda x:x.lower())

data4['corrupted']=data4['corrupted'].apply(lambda x:x.lower())
data4['uncorrupted']=data4['uncorrupted'].apply(lambda x:x.lower())

In [15]:
data['corrupted_len'] = data['corrupted'].str.split().apply(len)
data = data[data['corrupted_len'] < 25]

data['uncorrupted_len'] = data['uncorrupted'].str.split().apply(len)
data = data[data['uncorrupted_len'] < 25]

data4['corrupted_len'] = data4['corrupted'].str.split().apply(len)
data4 = data4[data4['corrupted_len'] < 25]

data4['uncorrupted_len'] = data4['uncorrupted'].str.split().apply(len)
data4 = data4[data4['uncorrupted_len'] < 25]

In [16]:
data['uncorrupted_inp']=data['uncorrupted'].apply(lambda x:('<START>'+' '+x))
data['uncorrupted_op']=data['uncorrupted'].apply(lambda x:(x+' '+'<END>'))
data.drop(['uncorrupted'],axis=1,inplace=True)

data4['uncorrupted_inp']=data4['uncorrupted'].apply(lambda x:('<START>'+' '+x))
data4['uncorrupted_op']=data4['uncorrupted'].apply(lambda x:(x+' '+'<END>'))
data4.drop(['uncorrupted'],axis=1,inplace=True)

In [17]:
data.drop(['corrupted_len','uncorrupted_len'],axis=1,inplace=True)
data4.drop(['corrupted_len','uncorrupted_len'],axis=1,inplace=True)

In [18]:
data=data.sample(5000)

In [19]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2)

In [20]:
train.shape

(4000, 3)

In [21]:
train=pd.concat([train,data4.iloc[0:1500]],ignore_index=True).astype(str)
validation=pd.concat([validation,data4.iloc[1500:1676]],ignore_index=True).astype(str)

In [22]:
train.head()

,corrupted,uncorrupted_inp,uncorrupted_op
0,it is also the reason why we have all these ne...,<START> that is also the reason why we have al...,that is also the reason why we have all these ...
1,my wife and me went to the circle theatre last...,<START> my wife and i went to the circle theat...,my wife and i went to the circle theatre last ...
2,but i am wondering why it said we could visit ...,<START> but i am wondering why it said we coul...,but i am wondering why it said we could visit ...
3,but it might not be the same everytime .,<START> but it might not be the same every time .,but it might not be the same every time . <END>
4,"that things had happend to everybody , had no...","<START> those things happen to everybody , do ...","those things happen to everybody , do not the..."


In [23]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['uncorrupted_inp']= str(train.iloc[0]['uncorrupted_inp'])+' <end>'
train.iloc[0]['uncorrupted_out']= str(train.iloc[0]['uncorrupted_op'])+' <end>'

(5500, 3) (1175, 3)


C:\Users\Shivam\AppData\Local\Temp\ipykernel_13084\3666877369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.iloc[0]['uncorrupted_inp']= str(train.iloc[0]['uncorrupted_inp'])+' <end>'
C:\Users\Shivam\AppData\Local\Temp\ipykernel_13084\3666877369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.iloc[0]['uncorrupted_out']= str(train.iloc[0]['uncorrupted_op'])+' <end>'
C:\Users\Shivam\AppData\Local\Temp\ipykernel_13084\3666877369.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [24]:
train.sample(frac=1)

,corrupted,uncorrupted_inp,uncorrupted_op
4030,ok...i try...coz i am still on é bus...is xin ...,<START> ok. i will try. it is because i am sti...,ok. i will try. it is because i am still on th...
2710,this is great opportunity for the student to l...,<START> this is a great opportunity for the st...,this is a great opportunity for the students t...
2884,"while i am at the camp , i would like to do t...","<START> while i am at the camp , i would like...","while i am at the camp , i would like to do t..."
908,i said she looked as if she did not want to t...,<START> i said . she looked as if she did not...,i said . she looked as if she did not want to...
3037,the main advantage of modern technology is get...,<START> the main advantage of modern technolog...,the main advantage of modern technology is get...
...,...,...,...
1268,these are some points i want mention about the...,<START> these are some points i want to mentio...,these are some points i want to mention about ...
51,i am writing to you to complain about last ev...,<START> i am writing to you to complain about...,i am writing to you to complain about last ev...
3631,first i would like to thank you for doing this...,<START> first i would like to thank you for or...,first i would like to thank you for organising...
4331,really how much...,"<START> really, how much?","really, how much? <END>"


In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok_corrupted=Tokenizer(filters='#$%&()*+-/=@[\\]^_`{|}~\t\n')
tok_corrupted.fit_on_texts(train['corrupted'].values)

tok_uncorrupted=Tokenizer(filters='#$%&()*+-/=@[\\]^_`{|}~\t\n')
tok_uncorrupted.fit_on_texts(train['uncorrupted_inp'].values)
tok_uncorrupted.fit_on_texts(train['uncorrupted_op'].values)

In [30]:
vocab_corrupt=len(tok_corrupted.word_index.keys())
print(vocab_corrupt)

vocab_uncorrupt=len(tok_uncorrupted.word_index.keys())
print(vocab_uncorrupt)

7594
6027
